In [1]:
import json

In [2]:
anno_json = '/Users/zhaolun/Documents/组学平台/bugs/task_10493_0.json'
anno_json = json.load(open(anno_json, 'r'))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/zhaolun/Documents/组学平台/bugs/task_10493_0.json'

In [ ]:
len(anno_json)

In [ ]:
anno = anno_json[2]
print(anno['nodes'][0])

In [ ]:
data_dir = '/Users/zhaolun/Documents/组学平台/bugs'

In [ ]:
import os
import SimpleITK as sitk
import numpy as np
import pydicom as dicom
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from skimage import measure

def draw_anno_3d(img_fn, anno, title = '', img_dir = None):
    img, dcm_files = read_dicom(img_fn, sort = True)

    img_arr = sitk.GetArrayFromImage(img)
    mask_arr = np.zeros_like(img_arr)

    for i, node in enumerate(anno['nodes']):
        rois = node["rois"]
        for roi in rois:
            slice_index = roi['slice_index'] -1
            points = roi['edge']
            for j in range(len(points)):
                points[j][0] = int(round(points[j][0]))
                points[j][1] = int(round(points[j][1]))
            points = np.array(points)
            cv2.drawContours(mask_arr[slice_index], (points,), -1, 1, thickness=-1)
    if img_dir:
        if mask_arr.sum() > 0:
            show_img_mask(img_arr, mask_arr, title, img_dir)
    return img_arr, mask_arr

def read_dicom(dicom_path, sort = True):
    if os.path.isfile(dicom_path):
        return sitk.ReadImage(dicom_path), dicom_path
    reader = sitk.ImageSeriesReader()
    dcm_files = list(reader.GetGDCMSeriesFileNames(dicom_path))
    if sort:
        dcm_files.sort(key = lambda x: dicom.read_file(x).InstanceNumber)

    reader.SetFileNames(dcm_files)
    img = reader.Execute()

    return img, dcm_files

def lumTrans(img, low = -600, high = 600, scale = 255):
    lungwin = np.array([low * 1., high * 1.])
    newimg = (img-lungwin[0])/(lungwin[1]-lungwin[0])
    newimg[newimg<0]=0
    newimg[newimg>1]=1
    newimg = (newimg*scale).astype('float32')
    return newimg

In [ ]:
img_fn = anno['other_info']['dataPath']
img_fn = img_fn.replace('/data/disk2/mediastinum/tjmu/lymph_node_enlargement', data_dir)

print(os.path.isdir(img_fn), img_fn)

In [ ]:
img_arr, mask_arr = draw_anno_3d(img_fn, anno)

In [ ]:
img_arr = lumTrans(img_arr, -250, 250)

In [ ]:
arrlist = []
for i in range(len(img_arr)):
    if mask_arr[i].max() > 0:
        fig, ax = plt.subplots(1,2, figsize = (16, 8))
        ax[0].imshow(img_arr[i], 'gray')
        contours = measure.find_contours(mask_arr[i], 0)
        for n, contour in enumerate(contours):
            ax[0].plot(contour[:, 1], contour[:, 0], linewidth=1)
        ax[1].imshow(mask_arr[i])
        arrlist.append(i)
        plt.show()
        plt.close()